<a href="https://colab.research.google.com/github/kumiori/mec647/blob/francoisfernet---mesh/generate_mesh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
import sys

try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl  # noqa: F401
    import dolfinx  # noqa: F401
else:
    try:
        import ufl
        import dolfinx
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenicsx-install.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh";
        import ufl  # noqa: F401
        import dolfinx  # noqa: F401

try:
    import pyvista
except ImportError:
    !{sys.executable} -m pip install --upgrade pyvista itkwidgets;
    import pyvista  # noqa: F401
    from pyvista.utilities import xvfb

try:
    import gmsh
except ImportError:
    !{sys.executable} -m pip install gmsh
    import gmsh

!sudo apt install libgl1-mesa-glx xvfb;
!{sys.executable} -m pip install pythreejs;
!{sys.executable} -m pip install ipygany;
!{sys.executable} -m pip install --upgrade PyYAML
# !pip install --ignore-installed PyYAML

In [6]:
branch_name = 'main'

!rm -rf mec647
try:
  !git clone -b {branch_name} https://github.com/kumiori/mec647.git
except Exception:
  print('Something went wrong')
  !rm -rf mec647
  !git clone https://github.com/kumiori/mec647.git


sys.path.append('mec647/')


import matplotlib.pyplot as plt
from petsc4py import PETSc
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD


Cloning into 'mec647'...
remote: Enumerating objects: 1032, done.
remote: Counting objects: 100% (1032/1032), done.
remote: Compressing objects: 100% (776/776), done.
remote: Total 1032 (delta 515), reused 531 (delta 230), pack-reused 0
Receiving objects: 100% (1032/1032), 18.49 MiB | 13.45 MiB/s, done.
Resolving deltas: 100% (515/515), done.


In [7]:
def  mesh_rh(name,
                     Lx,
                     Ly,
                     a,
                     b,
                     lc,
                     tdim,
                     order=1,
                     msh_file=None,
                     comm=MPI.COMM_WORLD):
    """
    Create mesh of 3d tensile test specimen according to ISO 6892-1:2019 using the Python API of Gmsh.
    """
    # Perform Gmsh work only on rank = 0

    if comm.rank == 0:

        import gmsh

        # Initialise gmsh and set options
        gmsh.initialize()
        gmsh.option.setNumber("General.Terminal", 1)

        gmsh.option.setNumber("Mesh.Algorithm", 6)
        model = gmsh.model()
        model.add("Rectangle")
        model.setCurrent("Rectangle")
        # points = [p1, p2, p3, p4, p5, p6, p7, p8,p9,p10,p11]
        p1 = model.geo.addPoint(0.0, 0.0, 0, lc, tag=1)
        p2 = model.geo.addPoint(Lx, 0.0, 0, lc, tag=2)
        p3 = model.geo.addPoint(Lx, Ly, 0.0, lc, tag=3)
        p4 = model.geo.addPoint(0, Ly, 0, lc, tag=4)
        p5 = model.geo.addPoint(Lx/2+b, Ly/2, 0.0, lc/2, tag=5)
        p6 = model.geo.addPoint(Lx/2, Ly/2+a, 0, lc/2, tag=6)
        p7 = model.geo.addPoint(Lx/2-b, Ly/2, 0.0, lc/2, tag=7)
        p8 = model.geo.addPoint(Lx/2, Ly/2-a, 0, lc/2 , tag=8)

        # Lines = [L1, L2, L3, L4, L5, L6, L7, L8]
        L1 = model.geo.addLine(p1, p2, tag=1)
        L2 = model.geo.addLine(p2, p3, tag=2)
        L3 = model.geo.addLine(p3, p4, tag=3)
        L4 = model.geo.addLine(p4, p1, tag=4)
        L5 = model.geo.addLine(p5, p6, tag=5)
        L6 = model.geo.addLine(p6, p7, tag=6)
        L7 = model.geo.addLine(p7, p8, tag=7)
        L8 = model.geo.addLine(p8, p5, tag=8)


        
        cloop1 = model.geo.addCurveLoop([L1, L2, L3, L4])
        cloop2 = model.geo.addCurveLoop([L5, L6, L7, L8])
        
        # surface_1 =
        model.geo.addPlaneSurface([cloop1, cloop2])

        model.geo.synchronize()
        surface_entities = [model[1] for model in model.getEntities(tdim)]

        model.addPhysicalGroup(tdim, surface_entities, tag=1)
        model.setPhysicalName(tdim, 1, "Rectangle surface")

        # Set mesh size via points
        # gmsh.model.mesh.setSize(points, lc)  # heuristic

        # gmsh.model.mesh.optimize("Netgen")

        # Set geometric order of mesh cells
        gmsh.model.mesh.setOrder(order)

        # Define physical groups for subdomains (! target tag > 0)
        # domain = 1
        # gmsh.model.addPhysicalGroup(tdim, [v[1] for v in volumes], domain)
        # gmsh.model.setPhysicalName(tdim, domain, 'domain')
              

        gmsh.model.addPhysicalGroup(tdim - 1, [1], tag=9)
        gmsh.model.setPhysicalName(tdim - 1, 9, "Bottom  of the Plate")
        gmsh.model.addPhysicalGroup(tdim - 1, [4], tag=10)
        gmsh.model.setPhysicalName(tdim - 1, 10, "Top  of the Plate")
        gmsh.model.addPhysicalGroup(tdim - 1, [5], tag=11)
        gmsh.model.setPhysicalName(tdim - 1, 11, "1/2 Top of the TipCrack")
        gmsh.model.addPhysicalGroup(tdim - 1, [6], tag=12)
        gmsh.model.setPhysicalName(tdim - 1, 12, "1/2 Top of the TipCrack")
        gmsh.model.addPhysicalGroup(tdim - 1, [7], tag=13)
        gmsh.model.setPhysicalName(tdim - 1, 13, "1/2 bottom of the TipCrack")
        gmsh.model.addPhysicalGroup(tdim - 1, [8], tag=14)
        gmsh.model.setPhysicalName(tdim - 1, 14, "1/2 bottom of the TipCrack")

        model.mesh.generate(tdim)


        #Generating the mesh
    model.geo.synchronize()
    model.mesh.generate(tdim)

    return gmsh.model

In [19]:
from random import seed 
import numpy as np 
import gmsh 
import matplotlib.pyplot as plt 
import meshes 
from meshes import primitives 
from utils.viz import plot_mesh#, plot_vector, plot_scalar


# Mesh
Lx = 100
Ly = 200
svals=np.arange(4, 35, step=3)
lvals=np.arange(5, 70, step=3)
for s in svals:
    for L0 in lvals:
        seedDist=1
        Lx=15
        Ly=25
        ss=2
        beta=np.pi/4
        lc=0.5

        a=ss* np.cos(beta/2)
        b=ss* np.sin(beta/2)

        print(a)
        print(b)

        gmsh_model=mesh_rh('mesh', Lx, Ly, a,b,lc, 2, 1)
    
        """
        # Only needed when we are generating a mesh to validate the calculations
        gmsh_model, tdim = primitives.mesh_rightCrack_gmshapi(geom_type,
                                            Lx, 
                                            Ly,
                                            L0, 
                                            s,   
                                            seedDist, 
                                            sep=1,
                                            tdim=2)
        """

        mesh, mts = meshes.gmsh_model_to_mesh(gmsh_model,
                                    cell_data=False,
                                    facet_data=True,
                                    gdim=2, 
                                    exportMesh=True, 
                                    #fileName="https://github.com/kumiori/mec647/tree/francoisfernet---mesh/meshes/"+str(s)+"_"+str(L0)+".unv")
                                    fileName="./meshes/"+str(s)+"_"+str(L0)+".unv")


        """plt.figure()
        ax = plot_mesh(mesh)
        fig = ax.get_figure()
        fig.savefig(f"mesh.png")"""

for s in svals:
    for L0 in lvals:
        print("mesh="+str(s)+"_"+str(L0)+";")

1.8477590650225735
0.7653668647301796


Exception: ignored